In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os 

os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/MyDrive/kaggle_dataset'

In [ ]:
%cd /content/gdrive/MyDrive/kaggle_dataset

/content/gdrive/MyDrive/kaggle_dataset


In [ ]:
%cd land-use

/content/gdrive/MyDrive/kaggle_dataset/land-use


In [ ]:
!nvidia-smi

Tue Apr 12 12:15:47 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
datagen = ImageDataGenerator(
        rescale = 1./255,
        horizontal_flip=True,
        vertical_flip=True)

img_size = 224
batch_size = 32

train_generator = datagen.flow_from_directory(
    'images_train_test_val/train',  # this is the target directory
    target_size=(img_size, img_size),  # all images will be resized to 300x300
    batch_size=batch_size,
    class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = datagen.flow_from_directory(
    'images_train_test_val/validation',
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical')

Found 7350 images belonging to 21 classes.
Found 2100 images belonging to 21 classes.


In [ ]:
from tensorflow.keras.applications import DenseNet201, VGG16, VGG19, ResNet50, InceptionV3, DenseNet201, Xception, MobileNetV2, EfficientNetB5
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
img_size = 224
base_model = VGG16(include_top = False,
                   weights = 'imagenet',
                   input_shape = (img_size,img_size,3))

for layer in base_model.layers[:-4]:
    layer.trainable = False 

last_layer = base_model.get_layer('block5_pool')
last_output = base_model.output

x = Flatten()(last_output)
x = Dense(21, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

58900480/58889256 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate = 0.0003), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_path = "vgg16.h5"
checkpoint = ModelCheckpoint(model_path,
                         monitor="val_loss",
                         mode="min",
                         save_best_only = True,
                         verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                      min_delta = 0, 
                      patience = 5,
                      verbose = 1,
                      restore_best_weights = True)

vgg16_history = model.fit(
                train_generator,
                steps_per_epoch= 7350 // batch_size,
                epochs=25,
                validation_data=validation_generator,
                validation_steps= 2100 // batch_size,
                callbacks=[checkpoint,earlystop])

Epoch 1/25
  9/229 [>.............................] - ETA: 39:40 - loss: 2.8308 - accuracy: 0.1528

KeyboardInterrupt: ignored

In [ ]:
img_size = 224
base_model = DenseNet201(include_top = False,
                        weights = 'imagenet',
                        input_shape = (img_size,img_size,3))

base_model.summary()

74850304/74836368 [==============================] - 0s 0us/step
Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_8[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

In [ ]:
base_model.layers[-4:]

In [ ]:
for layer in base_model.layers[:-9]:
    layer.trainable = False 

last_layer = base_model.get_layer('conv5_block32_concat')
last_output = base_model.output

x = GlobalAveragePooling2D()(last_output)
x = Dense(21, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

In [ ]:
model.compile(tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_path = "DenseNet201.h5"
checkpoint = ModelCheckpoint(model_path,
                         monitor="val_loss",
                         mode="min",
                         save_best_only = True,
                         verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                      min_delta = 0, 
                      patience = 5,
                      verbose = 1,
                      restore_best_weights = True)

history = model.fit(
                train_generator,
                steps_per_epoch= 7350 // batch_size,
                epochs=25,
                validation_data=validation_generator,
                validation_steps= 2100 // batch_size,
                callbacks=[checkpoint,earlystop])

Epoch 1/25
229/229 [==============================] - ETA: 0s - loss: 0.9279 - accuracy: 0.7769
Epoch 00001: val_loss improved from inf to 0.41188, saving model to DenseNet201.h5
229/229 [==============================] - 71s 270ms/step - loss: 0.9279 - accuracy: 0.7769 - val_loss: 0.4119 - val_accuracy: 0.9010
Epoch 2/25
229/229 [==============================] - ETA: 0s - loss: 0.2886 - accuracy: 0.9378
Epoch 00002: val_loss improved from 0.41188 to 0.27498, saving model to DenseNet201.h5
229/229 [==============================] - 58s 254ms/step - loss: 0.2886 - accuracy: 0.9378 - val_loss: 0.2750 - val_accuracy: 0.9236
Epoch 3/25
229/229 [==============================] - ETA: 0s - loss: 0.1974 - accuracy: 0.9585
Epoch 00003: val_loss improved from 0.27498 to 0.22217, saving model to DenseNet201.h5
229/229 [==============================] - 58s 255ms/step - loss: 0.1974 - accuracy: 0.9585 - val_loss: 0.2222 - val_accuracy: 0.9389
Epoch 4/25
229/229 [==============================] -

In [ ]:
datagen = ImageDataGenerator(
        rescale = 1./255)
test_generator = datagen.flow_from_directory('images_train_test_val/test',
    target_size=(img_size, img_size),
    batch_size=batch_size,              
    class_mode='categorical',
    shuffle=False)

Found 1050 images belonging to 21 classes.


In [ ]:
from sklearn.metrics import classification_report, f1_score, accuracy_score

In [ ]:
model = tf.keras.models.load_model('vgg16.h5')
Y_pred = model.predict(test_generator)

y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

target_names = []

for key in test_generator.class_indices:
    target_names.append(key)

print(classification_report(y_true,y_pred,target_names=target_names))
print(f1_score(y_true,y_pred,average='weighted'))
print(accuracy_score(y_true,y_pred))

                   precision    recall  f1-score   support

     agricultural       1.00      0.98      0.99        50
         airplane       0.93      1.00      0.96        50
  baseballdiamond       0.92      0.96      0.94        50
            beach       0.94      0.94      0.94        50
        buildings       0.90      0.90      0.90        50
        chaparral       1.00      1.00      1.00        50
 denseresidential       0.84      0.82      0.83        50
           forest       1.00      1.00      1.00        50
          freeway       0.94      0.98      0.96        50
       golfcourse       0.93      1.00      0.96        50
           harbor       0.98      1.00      0.99        50
     intersection       0.92      0.98      0.95        50
mediumresidential       0.93      0.80      0.86        50
   mobilehomepark       0.98      0.92      0.95        50
         overpass       1.00      0.94      0.97        50
       parkinglot       0.98      1.00      0.99       